# 🫀 Heart Segmentation Advanced - Model Architecture

<a href="https://colab.research.google.com/github/leonardobora/pratica-aprendizado-de-maquina/blob/main/Heart_Segmentation_Advanced/03_Model_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📋 Objetivos deste Notebook

Este notebook implementa arquiteturas avançadas de redes neurais para segmentação cardíaca:

- 🏗️ **U-Net Base Aprimorada** com melhorias modernas
- 🧠 **Backbones Pré-treinados** (ResNet, EfficientNet, DenseNet)
- ⚡ **Mecanismos de Atenção** (Attention Gates, Self-Attention)
- 🔧 **Normalização por Lotes** e Dropout
- 🎯 **Variantes Arquiteturais** (U-Net++, Attention U-Net)
- 📊 **Comparação de Modelos** e análise de complexidade

---

**⚠️ PRÉ-REQUISITOS**: 
- Execute `00_Setup_and_Configuration.ipynb`
- Execute `01_Data_Analysis_and_Preprocessing.ipynb`
- Execute `02_Data_Augmentation.ipynb`

In [2]:
# =============================================================================
# 📚 SETUP E CONFIGURAÇÕES
# =============================================================================

# Carregar configurações do projeto
import json
import os
import sys

try:
    with open('project_config.json', 'r') as f:
        project_config = json.load(f)
    print("✅ Configurações carregadas")
except FileNotFoundError:
    print("⚠️ Execute primeiro 00_Setup_and_Configuration.ipynb")
    sys.exit(1)

# Imports principais
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision.transforms as transforms
from torch.optim import Adam
import warnings
warnings.filterwarnings('ignore')

# Verificar GPU
print(f"🔧 PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Device: {device}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("📚 Setup completo!")

✅ Configurações carregadas
🔧 PyTorch version: 2.7.1+cpu
🚀 Device: cpu
📚 Setup completo!


In [3]:
# =============================================================================
# 🧱 BLOCOS BÁSICOS DE CONSTRUÇÃO
# =============================================================================

class ConvBlock(nn.Module):
    """Bloco de convolução básico com BatchNorm e Dropout"""
    
    def __init__(self, in_channels, out_channels, kernel_size=3, 
                 batch_norm=True, dropout_rate=0.0):
        super(ConvBlock, self).__init__()
        self.batch_norm = batch_norm
        self.dropout_rate = dropout_rate
        
        # Primeira convolução
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, 
                              padding=kernel_size//2, bias=not batch_norm)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 
                              padding=kernel_size//2, bias=not batch_norm)
        
        # Normalização
        if batch_norm:
            self.bn1 = nn.BatchNorm2d(out_channels)
            self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Dropout
        if dropout_rate > 0:
            self.dropout = nn.Dropout2d(dropout_rate)
        
        # Ativação
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # Primeira convolução
        out = self.conv1(x)
        if self.batch_norm:
            out = self.bn1(out)
        out = self.relu(out)
        
        if self.dropout_rate > 0:
            out = self.dropout(out)
        
        # Segunda convolução
        out = self.conv2(out)
        if self.batch_norm:
            out = self.bn2(out)
        out = self.relu(out)
        
        return out

class ResidualBlock(nn.Module):
    """Bloco residual com skip connection"""
    
    def __init__(self, in_channels, out_channels, kernel_size=3, 
                 batch_norm=True, dropout_rate=0.0):
        super(ResidualBlock, self).__init__()
        
        self.conv_block = ConvBlock(in_channels, out_channels, kernel_size, 
                                   batch_norm, dropout_rate)
        
        # Skip connection (1x1 conv se necessário)
        self.skip_conv = None
        if in_channels != out_channels:
            self.skip_conv = nn.Conv2d(in_channels, out_channels, 1, bias=False)
            if batch_norm:
                self.skip_bn = nn.BatchNorm2d(out_channels)
        
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # Main path
        out = self.conv_block(x)
        
        # Skip connection
        if self.skip_conv is not None:
            skip = self.skip_conv(x)
            if hasattr(self, 'skip_bn'):
                skip = self.skip_bn(skip)
        else:
            skip = x
        
        # Add skip connection
        out = out + skip
        return self.relu(out)

print("✅ Blocos básicos definidos")

✅ Blocos básicos definidos


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# =============================================================================
# ⚡ MECANISMOS DE ATENÇÃO
# =============================================================================

class AttentionGate(nn.Module):
    """Attention Gate para U-Net"""
    
    def __init__(self, F_g, F_l, F_int):
        """
        Args:
            F_g: número de canais do gating signal
            F_l: número de canais do feature map 
            F_int: número de canais intermediários
        """
        super(AttentionGate, self).__init__()
        
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, g, x):
        """
        Args:
            g: Gating signal do decoder
            x: Feature map do encoder
        """
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        
        return x * psi

class SelfAttention(nn.Module):
    """Self-Attention mechanism"""
    
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.in_channels = in_channels
        
        # Query, Key, Value projections
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, 1)
        
        # Learnable parameter
        self.gamma = nn.Parameter(torch.zeros(1))
        
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        """
        Args:
            x: input feature maps (B, C, H, W)
        Returns:
            out: self attention value + input feature
            attention: attention map (B, N, N)
        """
        B, C, H, W = x.size()
        N = H * W
        
        # Generate Q, K, V
        q = self.query_conv(x).view(B, -1, N).permute(0, 2, 1)  # B, N, C//8
        k = self.key_conv(x).view(B, -1, N)                     # B, C//8, N
        v = self.value_conv(x).view(B, -1, N)                   # B, C, N
        
        # Attention computation
        attention = torch.bmm(q, k)                              # B, N, N
        attention = self.softmax(attention)
        
        out = torch.bmm(v, attention.permute(0, 2, 1))          # B, C, N
        out = out.view(B, C, H, W)                              # B, C, H, W
        
        # Apply learnable parameter and residual connection
        out = self.gamma * out + x
        
        return out

class ChannelAttention(nn.Module):  
    """Channel Attention (SE Block)"""
    
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction_ratio, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction_ratio, in_channels, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

print("✅ Mecanismos de atenção definidos")

✅ Mecanismos de atenção definidos


In [5]:
# =============================================================================
# 🏗️ U-NET BASE APRIMORADA
# =============================================================================

class EnhancedUNet(nn.Module):
    """U-Net aprimorada com opções modernas"""
    
    def __init__(self, in_channels=1, num_classes=3, filters_base=64, 
                 use_attention=True, use_residual=False, dropout_rate=0.1):
        super(EnhancedUNet, self).__init__()
        
        self.use_attention = use_attention
        self.use_residual = use_residual
        
        # Encoder
        if use_residual:
            self.encoder1 = ResidualBlock(in_channels, filters_base, dropout_rate=dropout_rate)
            self.encoder2 = ResidualBlock(filters_base, filters_base*2, dropout_rate=dropout_rate)  
            self.encoder3 = ResidualBlock(filters_base*2, filters_base*4, dropout_rate=dropout_rate)
            self.encoder4 = ResidualBlock(filters_base*4, filters_base*8, dropout_rate=dropout_rate)
        else:
            self.encoder1 = ConvBlock(in_channels, filters_base, dropout_rate=dropout_rate)
            self.encoder2 = ConvBlock(filters_base, filters_base*2, dropout_rate=dropout_rate)
            self.encoder3 = ConvBlock(filters_base*2, filters_base*4, dropout_rate=dropout_rate)
            self.encoder4 = ConvBlock(filters_base*4, filters_base*8, dropout_rate=dropout_rate)
        
        # Bottleneck
        self.bottleneck = ConvBlock(filters_base*8, filters_base*16, dropout_rate=dropout_rate)
        
        # Decoder upsampling
        self.upconv4 = nn.ConvTranspose2d(filters_base*16, filters_base*8, 2, stride=2)
        self.upconv3 = nn.ConvTranspose2d(filters_base*8, filters_base*4, 2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(filters_base*4, filters_base*2, 2, stride=2)
        self.upconv1 = nn.ConvTranspose2d(filters_base*2, filters_base, 2, stride=2)
        
        # Decoder conv blocks
        if use_residual:
            self.decoder4 = ResidualBlock(filters_base*16, filters_base*8, dropout_rate=dropout_rate)
            self.decoder3 = ResidualBlock(filters_base*8, filters_base*4, dropout_rate=dropout_rate)
            self.decoder2 = ResidualBlock(filters_base*4, filters_base*2, dropout_rate=dropout_rate)
            self.decoder1 = ResidualBlock(filters_base*2, filters_base, dropout_rate=dropout_rate)
        else:
            self.decoder4 = ConvBlock(filters_base*16, filters_base*8, dropout_rate=dropout_rate)
            self.decoder3 = ConvBlock(filters_base*8, filters_base*4, dropout_rate=dropout_rate)
            self.decoder2 = ConvBlock(filters_base*4, filters_base*2, dropout_rate=dropout_rate)
            self.decoder1 = ConvBlock(filters_base*2, filters_base, dropout_rate=dropout_rate)
        
        # Attention gates
        if use_attention:
            self.att4 = AttentionGate(filters_base*8, filters_base*8, filters_base*4)
            self.att3 = AttentionGate(filters_base*4, filters_base*4, filters_base*2)
            self.att2 = AttentionGate(filters_base*2, filters_base*2, filters_base)
            self.att1 = AttentionGate(filters_base, filters_base, filters_base//2)
        
        # Max pooling
        self.pool = nn.MaxPool2d(2)
        
        # Final classifier
        self.final = nn.Conv2d(filters_base, num_classes, 1)
        
        print(f"✅ Enhanced U-Net criada:")
        print(f"   📊 Canais entrada: {in_channels}")
        print(f"   🏷️ Classes: {num_classes}")
        print(f"   🔧 Filtros base: {filters_base}")
        print(f"   ⚡ Atenção: {use_attention}")
        print(f"   🔗 Residual: {use_residual}")
        print(f"   💧 Dropout: {dropout_rate}")
    
    def forward(self, x):
        # Encoder
        enc1 = self.encoder1(x)          # N, 64, H, W
        enc2 = self.encoder2(self.pool(enc1))   # N, 128, H/2, W/2
        enc3 = self.encoder3(self.pool(enc2))   # N, 256, H/4, W/4
        enc4 = self.encoder4(self.pool(enc3))   # N, 512, H/8, W/8
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))  # N, 1024, H/16, W/16
        
        # Decoder
        dec4 = self.upconv4(bottleneck)  # N, 512, H/8, W/8
        if self.use_attention:
            enc4 = self.att4(dec4, enc4)
        dec4 = torch.cat([dec4, enc4], dim=1)  # N, 1024, H/8, W/8
        dec4 = self.decoder4(dec4)       # N, 512, H/8, W/8
        
        dec3 = self.upconv3(dec4)        # N, 256, H/4, W/4
        if self.use_attention:
            enc3 = self.att3(dec3, enc3)
        dec3 = torch.cat([dec3, enc3], dim=1)  # N, 512, H/4, W/4
        dec3 = self.decoder3(dec3)       # N, 256, H/4, W/4
        
        dec2 = self.upconv2(dec3)        # N, 128, H/2, W/2
        if self.use_attention:
            enc2 = self.att2(dec2, enc2)
        dec2 = torch.cat([dec2, enc2], dim=1)  # N, 256, H/2, W/2
        dec2 = self.decoder2(dec2)       # N, 128, H/2, W/2
        
        dec1 = self.upconv1(dec2)        # N, 64, H, W
        if self.use_attention:
            enc1 = self.att1(dec1, enc1)
        dec1 = torch.cat([dec1, enc1], dim=1)  # N, 128, H, W
        dec1 = self.decoder1(dec1)       # N, 64, H, W
        
        # Final output
        output = self.final(dec1)        # N, num_classes, H, W
        
        return output

def create_enhanced_unet(in_channels=1, num_classes=3, filters_base=64, 
                        use_attention=True, use_residual=False, dropout_rate=0.1):
    """Factory function para criar Enhanced U-Net"""
    return EnhancedUNet(in_channels, num_classes, filters_base, 
                       use_attention, use_residual, dropout_rate)

# Teste da arquitetura
print("\n🧪 TESTANDO ARQUITETURA")
print("=" * 50)

model = create_enhanced_unet()
print(f"📊 Parâmetros totais: {sum(p.numel() for p in model.parameters()):,}")
print(f"📊 Parâmetros treináveis: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Teste com entrada dummy
test_input = torch.randn(1, 1, 128, 128)
print(f"🔍 Input shape: {test_input.shape}")

with torch.no_grad():
    test_output = model(test_input)
    print(f"🎯 Output shape: {test_output.shape}")

print("✅ Teste de arquitetura concluído!")


🧪 TESTANDO ARQUITETURA
✅ Enhanced U-Net criada:
   📊 Canais entrada: 1
   🏷️ Classes: 3
   🔧 Filtros base: 64
   ⚡ Atenção: True
   🔗 Residual: False
   💧 Dropout: 0.1
📊 Parâmetros totais: 31,388,143
📊 Parâmetros treináveis: 31,388,143
🔍 Input shape: torch.Size([1, 1, 128, 128])
🎯 Output shape: torch.Size([1, 3, 128, 128])
✅ Teste de arquitetura concluído!


In [ ]:
# =============================================================================
# 🧠 BACKBONES PRÉ-TREINADOS
# =============================================================================

def create_backbone_encoder(backbone_name='resnet50', input_shape=(128, 128, 3)):
    """
    Cria encoder baseado em backbone pré-treinado
    
    Args:
        backbone_name: Nome do backbone ('resnet50', 'efficientnet-b0', 'densenet121')
        input_shape: Forma da entrada
    
    Returns:
        Tuple: (encoder_model, skip_connections_names)
    """
    
    if backbone_name == 'resnet50':
        backbone = ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
        
        # Camadas para skip connections
        skip_layers = [
            'conv1_relu',           # 64x64
            'conv2_block3_out',     # 32x32  
            'conv3_block4_out',     # 16x16
            'conv4_block6_out',     # 8x8
            'conv5_block3_out'      # 4x4
        ]
        
    elif backbone_name == 'efficientnet-b0':
        backbone = EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
        
        # Camadas para skip connections (EfficientNet)
        skip_layers = [
            'block2a_expand_activation',  # 64x64
            'block3a_expand_activation',  # 32x32
            'block4a_expand_activation',  # 16x16
            'block6a_expand_activation',  # 8x8
            'top_activation'              # 4x4
        ]
        
    elif backbone_name == 'densenet121':
        backbone = DenseNet121(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
        
        # Camadas para skip connections (DenseNet)
        skip_layers = [
            'conv1/relu',                    # 64x64
            'pool2_pool',                    # 32x32
            'pool3_pool',                    # 16x16
            'pool4_pool',                    # 8x8
            'relu'                           # 4x4
        ]
    
    else:
        raise ValueError(f"Backbone não suportado: {backbone_name}")
    
    # Freezar algumas camadas iniciais
    for layer in backbone.layers[:50]:  # Freezar primeiras 50 camadas
        layer.trainable = False
    
    print(f"🧠 Backbone {backbone_name} carregado")
    print(f"   Parâmetros totais: {backbone.count_params():,}")
    print(f"   Parâmetros treináveis: {sum([tf.keras.utils.count_params(w) for w in backbone.trainable_weights]):,}")
    
    return backbone, skip_layers

def create_unet_with_backbone(backbone_name='resnet50', input_shape=(128, 128, 1), 
                             num_classes=3, use_attention=True):
    """
    Cria U-Net com backbone pré-treinado
    
    Args:
        backbone_name: Nome do backbone
        input_shape: Forma da entrada
        num_classes: Número de classes
        use_attention: Usar attention gates
    
    Returns:
        Model: Modelo Keras
    """
    
    # Ajustar input para 3 canais se necessário
    if input_shape[-1] == 1:
        backbone_input_shape = (*input_shape[:2], 3)
        
        # Input original
        inputs = Input(shape=input_shape, name='input')
        
        # Converter para 3 canais (repetir canal)
        x = layers.Concatenate()([inputs, inputs, inputs])
    else:
        backbone_input_shape = input_shape
        inputs = Input(shape=input_shape, name='input')
        x = inputs
    
    # Criar backbone encoder
    backbone, skip_layers = create_backbone_encoder(backbone_name, backbone_input_shape)
    
    # Extrair features em diferentes escalas
    skip_features = []
    x = backbone(x)
    
    # Para este exemplo, vamos usar uma abordagem simplificada
    # Criando features artificiais para demonstração
    print("🔧 Construindo decoder com backbone...")
    
    # Decoder simplificado (versão demonstrativa)
    # Nivel 4 (4x4 -> 8x8)
    x = layers.UpSampling2D(2, interpolation='bilinear')(x)
    x = ConvBlock(512, name='decoder_4')(x)
    
    # Nivel 3 (8x8 -> 16x16) 
    x = layers.UpSampling2D(2, interpolation='bilinear')(x)
    x = ConvBlock(256, name='decoder_3')(x)
    
    # Nivel 2 (16x16 -> 32x32)
    x = layers.UpSampling2D(2, interpolation='bilinear')(x)
    x = ConvBlock(128, name='decoder_2')(x)
    
    # Nivel 1 (32x32 -> 64x64)
    x = layers.UpSampling2D(2, interpolation='bilinear')(x)
    x = ConvBlock(64, name='decoder_1')(x)
    
    # Final (64x64 -> 128x128)
    x = layers.UpSampling2D(2, interpolation='bilinear')(x)
    x = ConvBlock(32, name='decoder_0')(x)
    
    # Output
    outputs = layers.Conv2D(num_classes, 1, activation='softmax', name='output')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name=f'UNet_{backbone_name}')
    
    print(f"✅ U-Net com {backbone_name} criada: {model.count_params():,} parâmetros")
    return model

# Testar backbones
print("🧪 Testando backbones pré-treinados...")

try:
    # U-Net com ResNet50
    unet_resnet = create_unet_with_backbone(
        backbone_name='resnet50',
        input_shape=(128, 128, 1),
        num_classes=3
    )
    
    # U-Net com EfficientNet-B0
    unet_efficientnet = create_unet_with_backbone(
        backbone_name='efficientnet-b0',
        input_shape=(128, 128, 1),
        num_classes=3
    )
    
    print("✅ Modelos com backbones criados com sucesso!")
    
except Exception as e:
    print(f"⚠️ Erro ao criar modelos com backbone: {e}")
    print("Continuando com modelos básicos...")

In [ ]:
# =============================================================================
# 🎯 U-NET++ (NESTED U-NET)
# =============================================================================

def create_unet_plusplus(input_shape=(128, 128, 1), num_classes=3, 
                        filters_base=64, deep_supervision=True):
    """
    Implementa U-Net++ (Nested U-Net)
    
    Args:
        input_shape: Forma da entrada
        num_classes: Número de classes
        filters_base: Número base de filtros
        deep_supervision: Usar supervisão profunda
    
    Returns:
        Model: Modelo U-Net++
    """
    
    inputs = Input(shape=input_shape, name='input')
    
    # Encoder pathway
    print("🔧 Construindo U-Net++ Encoder...")
    
    # Level 0 (128x128)
    x00 = ConvBlock(filters_base, name='conv_0_0')(inputs)
    pool0 = layers.MaxPooling2D(2)(x00)
    
    # Level 1 (64x64)
    x10 = ConvBlock(filters_base*2, name='conv_1_0')(pool0)
    pool1 = layers.MaxPooling2D(2)(x10)
    
    # Level 2 (32x32)
    x20 = ConvBlock(filters_base*4, name='conv_2_0')(pool1)
    pool2 = layers.MaxPooling2D(2)(x20)
    
    # Level 3 (16x16)
    x30 = ConvBlock(filters_base*8, name='conv_3_0')(pool2)
    pool3 = layers.MaxPooling2D(2)(x30)
    
    # Level 4 (8x8) - Bottleneck
    x40 = ConvBlock(filters_base*16, name='conv_4_0')(pool3)
    
    # Nested pathways
    print("🔧 Construindo U-Net++ Nested Pathways...")
    
    # Nested Level 0
    up_3_1 = layers.UpSampling2D(2, interpolation='bilinear')(x40)
    concat_3_1 = layers.Concatenate()([x30, up_3_1])
    x31 = ConvBlock(filters_base*8, name='conv_3_1')(concat_3_1)
    
    # Nested Level 1
    up_2_1 = layers.UpSampling2D(2, interpolation='bilinear')(x31)
    concat_2_1 = layers.Concatenate()([x20, up_2_1])
    x21 = ConvBlock(filters_base*4, name='conv_2_1')(concat_2_1)
    
    up_2_2 = layers.UpSampling2D(2, interpolation='bilinear')(x30)
    concat_2_2 = layers.Concatenate()([x20, x21, up_2_2])
    x22 = ConvBlock(filters_base*4, name='conv_2_2')(concat_2_2)
    
    # Nested Level 2
    up_1_1 = layers.UpSampling2D(2, interpolation='bilinear')(x21)
    concat_1_1 = layers.Concatenate()([x10, up_1_1])
    x11 = ConvBlock(filters_base*2, name='conv_1_1')(concat_1_1)
    
    up_1_2 = layers.UpSampling2D(2, interpolation='bilinear')(x22)
    concat_1_2 = layers.Concatenate()([x10, x11, up_1_2])
    x12 = ConvBlock(filters_base*2, name='conv_1_2')(concat_1_2)
    
    up_1_3 = layers.UpSampling2D(2, interpolation='bilinear')(x31)
    concat_1_3 = layers.Concatenate()([x10, x11, x12, up_1_3])
    x13 = ConvBlock(filters_base*2, name='conv_1_3')(concat_1_3)
    
    # Final level
    up_0_1 = layers.UpSampling2D(2, interpolation='bilinear')(x11)
    concat_0_1 = layers.Concatenate()([x00, up_0_1])
    x01 = ConvBlock(filters_base, name='conv_0_1')(concat_0_1)
    
    up_0_2 = layers.UpSampling2D(2, interpolation='bilinear')(x12)
    concat_0_2 = layers.Concatenate()([x00, x01, up_0_2])
    x02 = ConvBlock(filters_base, name='conv_0_2')(concat_0_2)
    
    up_0_3 = layers.UpSampling2D(2, interpolation='bilinear')(x13)
    concat_0_3 = layers.Concatenate()([x00, x01, x02, up_0_3])
    x03 = ConvBlock(filters_base, name='conv_0_3')(concat_0_3)
    
    up_0_4 = layers.UpSampling2D(2, interpolation='bilinear')(x31)
    concat_0_4 = layers.Concatenate()([x00, x01, x02, x03, up_0_4])
    x04 = ConvBlock(filters_base, name='conv_0_4')(concat_0_4)
    
    # Outputs
    if deep_supervision:
        # Múltiplas saídas para supervisão profunda
        out1 = layers.Conv2D(num_classes, 1, activation='softmax', name='output_1')(x01)
        out2 = layers.Conv2D(num_classes, 1, activation='softmax', name='output_2')(x02)
        out3 = layers.Conv2D(num_classes, 1, activation='softmax', name='output_3')(x03)
        out4 = layers.Conv2D(num_classes, 1, activation='softmax', name='output_4')(x04)
        
        model = Model(inputs=inputs, outputs=[out1, out2, out3, out4], name='UNet_PlusPlus')
    else:
        # Saída única
        output = layers.Conv2D(num_classes, 1, activation='softmax', name='output')(x04)
        model = Model(inputs=inputs, outputs=output, name='UNet_PlusPlus')
    
    print(f"✅ U-Net++ criada: {model.count_params():,} parâmetros")
    return model

# Testar U-Net++
print("🧪 Testando U-Net++...")

try:
    unet_plusplus = create_unet_plusplus(
        input_shape=(128, 128, 1),
        num_classes=3,
        deep_supervision=False  # Simplificado para teste
    )
    print("✅ U-Net++ criada com sucesso!")
except Exception as e:
    print(f"⚠️ Erro ao criar U-Net++: {e}")

In [ ]:
# =============================================================================
# 📊 COMPARAÇÃO DE MODELOS
# =============================================================================

def analyze_model_complexity():
    """Analisa complexidade dos modelos criados"""
    
    print("📊 ANÁLISE DE COMPLEXIDADE DOS MODELOS")
    print("=" * 60)
    
    models_info = []
    
    # Lista de modelos para analisar
    models_to_analyze = [
        ("U-Net Básica", unet_basic),
        ("U-Net + Atenção", unet_attention), 
        ("U-Net + Atenção + Residual", unet_full),
    ]
    
    # Adicionar modelos com backbone se disponíveis
    try:
        models_to_analyze.extend([
            ("U-Net + ResNet50", unet_resnet),
            ("U-Net + EfficientNet-B0", unet_efficientnet),
        ])
    except:
        pass
    
    # Adicionar U-Net++ se disponível
    try:
        models_to_analyze.append(("U-Net++", unet_plusplus))
    except:
        pass
    
    # Analisar cada modelo
    for name, model in models_to_analyze:
        try:
            total_params = model.count_params()
            trainable_params = sum([tf.keras.utils.count_params(w) for w in model.trainable_weights])
            non_trainable_params = total_params - trainable_params
            
            # Calcular FLOPs (aproximado)
            input_shape = model.input_shape[1:]
            flops = estimate_flops(model, input_shape)
            
            # Calcular tamanho do modelo em MB
            model_size_mb = (total_params * 4) / (1024 * 1024)  # 4 bytes por parâmetro
            
            info = {
                'name': name,
                'total_params': total_params,
                'trainable_params': trainable_params,
                'non_trainable_params': non_trainable_params,
                'model_size_mb': model_size_mb,
                'flops': flops
            }
            
            models_info.append(info)
            
            print(f"\n🏗️ {name}:")
            print(f"   📊 Parâmetros totais: {total_params:,}")
            print(f"   🔧 Parâmetros treináveis: {trainable_params:,}")
            print(f"   🔒 Parâmetros não-treináveis: {non_trainable_params:,}")
            print(f"   💾 Tamanho do modelo: {model_size_mb:.2f} MB")
            print(f"   ⚡ FLOPs estimados: {flops:,}")
            
        except Exception as e:
            print(f"❌ Erro ao analisar {name}: {e}")
    
    # Criar visualização comparativa
    visualize_model_comparison(models_info)
    
    return models_info

def estimate_flops(model, input_shape):
    """Estima FLOPs do modelo (aproximação simples)"""
    total_flops = 0
    
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            # FLOPs para Conv2D = (kernel_size^2 * input_channels * output_channels * output_height * output_width)
            if hasattr(layer, 'kernel_size') and hasattr(layer, 'filters'):
                kernel_h, kernel_w = layer.kernel_size
                output_channels = layer.filters
                
                # Aproximação grosseira
                flops_per_conv = kernel_h * kernel_w * output_channels * (input_shape[0] * input_shape[1])
                total_flops += flops_per_conv
        
        elif isinstance(layer, layers.Dense):
            if hasattr(layer, 'units'):
                # FLOPs para Dense = input_size * output_size
                total_flops += layer.units * 1000  # Aproximação
    
    return total_flops

def visualize_model_comparison(models_info):
    """Visualiza comparação entre modelos"""
    
    if not models_info:
        print("⚠️ Nenhum modelo para comparar")
        return
    
    names = [info['name'] for info in models_info]
    total_params = [info['total_params'] for info in models_info]
    model_sizes = [info['model_size_mb'] for info in models_info]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('📊 Comparação de Complexidade dos Modelos', fontsize=16, fontweight='bold')
    
    # Gráfico 1: Número de parâmetros
    axes[0, 0].bar(names, total_params, color='skyblue', alpha=0.7)
    axes[0, 0].set_title('Número Total de Parâmetros')
    axes[0, 0].set_ylabel('Parâmetros')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Adicionar valores nas barras
    for i, v in enumerate(total_params):
        axes[0, 0].text(i, v, f'{v:,.0f}', ha='center', va='bottom')
    
    # Gráfico 2: Tamanho do modelo
    axes[0, 1].bar(names, model_sizes, color='lightcoral', alpha=0.7)
    axes[0, 1].set_title('Tamanho do Modelo')
    axes[0, 1].set_ylabel('Tamanho (MB)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Adicionar valores nas barras
    for i, v in enumerate(model_sizes):
        axes[0, 1].text(i, v, f'{v:.1f}MB', ha='center', va='bottom')
    
    # Gráfico 3: Parâmetros treináveis vs não-treináveis
    trainable = [info['trainable_params'] for info in models_info]
    non_trainable = [info['non_trainable_params'] for info in models_info]
    
    x = np.arange(len(names))
    width = 0.35
    
    axes[1, 0].bar(x - width/2, trainable, width, label='Treináveis', color='green', alpha=0.7)
    axes[1, 0].bar(x + width/2, non_trainable, width, label='Não-treináveis', color='red', alpha=0.7)
    axes[1, 0].set_title('Parâmetros Treináveis vs Não-treináveis')
    axes[1, 0].set_ylabel('Parâmetros')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(names, rotation=45)
    axes[1, 0].legend()
    
    # Gráfico 4: Efficiency score (parâmetros / MB)
    efficiency = [params / size if size > 0 else 0 for params, size in zip(total_params, model_sizes)]
    axes[1, 1].bar(names, efficiency, color='gold', alpha=0.7)
    axes[1, 1].set_title('Eficiência (Parâmetros por MB)')
    axes[1, 1].set_ylabel('Parâmetros/MB')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

# Executar análise
models_info = analyze_model_complexity()

In [ ]:
# =============================================================================
# 🎯 TESTE DE INFERÊNCIA DOS MODELOS
# =============================================================================

def test_model_inference():
    """Testa inferência dos modelos criados"""
    
    print("🎯 TESTANDO INFERÊNCIA DOS MODELOS")
    print("=" * 50)
    
    # Criar dados de teste
    batch_size = 2
    test_input = tf.random.normal([batch_size, 128, 128, 1])
    
    models_to_test = [
        ("U-Net Básica", unet_basic),
        ("U-Net + Atenção", unet_attention),
        ("U-Net + Atenção + Residual", unet_full),
    ]
    
    # Adicionar outros modelos se disponíveis
    try:
        models_to_test.extend([
            ("U-Net + ResNet50", unet_resnet),
            ("U-Net + EfficientNet-B0", unet_efficientnet),
        ])
    except:
        pass
    
    try:
        models_to_test.append(("U-Net++", unet_plusplus))
    except:
        pass
    
    inference_results = []
    
    for name, model in models_to_test:
        try:
            print(f"\n🔄 Testando {name}...")
            
            # Medir tempo de inferência
            import time
            start_time = time.time()
            
            predictions = model(test_input, training=False)
            
            end_time = time.time()
            inference_time = end_time - start_time
            
            # Verificar forma da saída
            if isinstance(predictions, list):
                output_shape = predictions[0].shape
                print(f"   📤 Saída múltipla: {len(predictions)} outputs")
                print(f"   📐 Forma principal: {output_shape}")
            else:
                output_shape = predictions.shape
                print(f"   📐 Forma da saída: {output_shape}")
            
            print(f"   ⏱️ Tempo de inferência: {inference_time:.4f}s")
            print(f"   🚀 FPS (aprox): {batch_size/inference_time:.2f}")
            
            # Verificar range de valores
            if isinstance(predictions, list):
                pred_to_check = predictions[0]
            else:
                pred_to_check = predictions
                
            print(f"   📊 Range de valores: [{tf.reduce_min(pred_to_check):.4f}, {tf.reduce_max(pred_to_check):.4f}]")
            print(f"   📈 Soma softmax (deve ser ~1): {tf.reduce_mean(tf.reduce_sum(pred_to_check, axis=-1)):.4f}")
            
            inference_results.append({
                'name': name,
                'inference_time': inference_time,
                'fps': batch_size/inference_time,
                'output_shape': output_shape,
                'success': True
            })
            
            print(f"   ✅ Teste bem-sucedido!")
            
        except Exception as e:
            print(f"   ❌ Erro durante inferência: {e}")
            inference_results.append({
                'name': name,
                'success': False,
                'error': str(e)
            })
    
    # Visualizar resultados de performance
    visualize_inference_performance(inference_results)
    
    return inference_results

def visualize_inference_performance(results):
    """Visualiza performance de inferência"""
    
    successful_results = [r for r in results if r.get('success', False)]
    
    if not successful_results:
        print("⚠️ Nenhum resultado de inferência bem-sucedido")
        return
    
    names = [r['name'] for r in successful_results]
    inference_times = [r['inference_time'] for r in successful_results]
    fps_values = [r['fps'] for r in successful_results]
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('🎯 Performance de Inferência dos Modelos', fontsize=16, fontweight='bold')
    
    # Tempo de inferência
    bars1 = axes[0].bar(names, inference_times, color='lightblue', alpha=0.7)
    axes[0].set_title('Tempo de Inferência')
    axes[0].set_ylabel('Tempo (segundos)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Adicionar valores nas barras
    for bar, time_val in zip(bars1, inference_times):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{time_val:.4f}s', ha='center', va='bottom')
    
    # FPS
    bars2 = axes[1].bar(names, fps_values, color='lightgreen', alpha=0.7)
    axes[1].set_title('Frames por Segundo (FPS)')
    axes[1].set_ylabel('FPS')
    axes[1].tick_params(axis='x', rotation=45)
    
    # Adicionar valores nas barras
    for bar, fps_val in zip(bars2, fps_values):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{fps_val:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# Executar teste de inferência
inference_results = test_model_inference()

In [ ]:
# =============================================================================
# 💾 SALVAR ARQUITETURAS DOS MODELOS
# =============================================================================

def save_model_architectures():
    """Salva arquiteturas dos modelos"""
    
    print("💾 SALVANDO ARQUITETURAS DOS MODELOS")
    print("=" * 50)
    
    models_path = project_config['paths']['models']
    os.makedirs(models_path, exist_ok=True)
    
    models_to_save = [
        ("unet_basic", unet_basic),
        ("unet_attention", unet_attention),
        ("unet_full", unet_full),
    ]
    
    # Adicionar outros modelos se disponíveis
    try:
        models_to_save.extend([
            ("unet_resnet50", unet_resnet),
            ("unet_efficientnet", unet_efficientnet),
        ])
    except:
        pass
    
    try:
        models_to_save.append(("unet_plusplus", unet_plusplus))
    except:
        pass
    
    for name, model in models_to_save:
        try:
            # Salvar arquitetura em JSON
            model_json = model.to_json()
            json_path = os.path.join(models_path, f"{name}_architecture.json")
            
            with open(json_path, 'w') as f:
                f.write(model_json)
            
            # Salvar summary em texto
            summary_path = os.path.join(models_path, f"{name}_summary.txt")
            
            with open(summary_path, 'w') as f:
                model.summary(print_fn=lambda x: f.write(x + '\n'))
            
            print(f"✅ {name}: Arquitetura e sumário salvos")
            
        except Exception as e:
            print(f"❌ Erro ao salvar {name}: {e}")
    
    # Salvar comparação de modelos
    try:
        comparison_data = {
            'models_comparison': models_info,
            'inference_results': inference_results,
            'created_at': str(pd.Timestamp.now()),
            'tensorflow_version': tf.__version__
        }
        
        comparison_path = os.path.join(models_path, 'models_comparison.json')
        with open(comparison_path, 'w') as f:
            json.dump(comparison_data, f, indent=2, default=str)
        
        print(f"✅ Comparação de modelos salva em: {comparison_path}")
        
    except Exception as e:
        print(f"❌ Erro ao salvar comparação: {e}")

# Salvar arquiteturas
save_model_architectures()

In [ ]:
# =============================================================================
# 📋 RECOMENDAÇÕES DE MODELOS
# =============================================================================

def generate_model_recommendations():
    """Gera recomendações de uso para cada modelo"""
    
    print("📋 RECOMENDAÇÕES DE USO DOS MODELOS")
    print("=" * 60)
    
    recommendations = {
        "U-Net Básica": {
            "uso_recomendado": "Baseline e prototipagem rápida",
            "vantagens": [
                "Simples e fácil de treinar",
                "Menor uso de memória",
                "Treinamento rápido"
            ],
            "desvantagens": [
                "Performance limitada",
                "Menos refinamento nas bordas"
            ],
            "cenarios": [
                "Datasets pequenos",
                "Prototipagem",
                "Recursos computacionais limitados"
            ]
        },
        
        "U-Net + Atenção": {
            "uso_recomendado": "Segmentação de alta qualidade",
            "vantagens": [
                "Melhor detecção de bordas",
                "Foco em regiões relevantes",
                "Boa performance geral"
            ],
            "desvantagens": [
                "Ligeiramente mais complexa",
                "Maior uso de memória"
            ],
            "cenarios": [
                "Aplicações clínicas",
                "Quando precisão é crítica",
                "Estruturas pequenas/complexas"
            ]
        },
        
        "U-Net + Atenção + Residual": {
            "uso_recomendado": "Máxima performance",
            "vantagens": [
                "Melhor propagação de gradientes",
                "Performance superior",
                "Estabilidade de treinamento"
            ],
            "desvantagens": [
                "Mais complexa",
                "Maior tempo de treinamento"
            ],
            "cenarios": [
                "Datasets grandes",
                "Máxima precisão necessária",
                "Recursos computacionais abundantes"
            ]
        }
    }
    
    # Adicionar recomendações para modelos com backbone
    try:
        recommendations["U-Net + ResNet50"] = {
            "uso_recomendado": "Transfer learning e features robustas",
            "vantagens": [
                "Features pré-treinadas",
                "Convergência mais rápida",
                "Boa generalização"
            ],
            "desvantagens": [
                "Modelo grande",
                "Requer mais memória"
            ],
            "cenarios": [
                "Datasets médios/grandes",
                "Transfer learning",
                "Quando features naturais ajudam"
            ]
        }
        
        recommendations["U-Net + EfficientNet-B0"] = {
            "uso_recomendado": "Eficiência computacional",
            "vantagens": [
                "Boa relação performance/eficiência",
                "Otimizado para mobile/edge",
                "Features modernas"
            ],
            "desvantagens": [
                "Menos investigado em medicina",
                "Potencialmente menos estável"
            ],
            "cenarios": [
                "Deploy em produção",
                "Recursos limitados",
                "Aplicações em tempo real"
            ]
        }
    except:
        pass
    
    # Adicionar recomendação para U-Net++
    try:
        recommendations["U-Net++"] = {
            "uso_recomendado": "Máxima precisão com supervisão profunda",
            "vantagens": [
                "Múltiplos paths de informação",
                "Supervisão em múltiplos níveis",
                "Performance state-of-the-art"
            ],
            "desvantagens": [
                "Muito complexa",
                "Treinamento lento",
                "Alto uso de memória"
            ],
            "cenarios": [
                "Pesquisa acadêmica",
                "Quando precisão é fundamental",
                "Datasets muito grandes"
            ]
        }
    except:
        pass
    
    # Exibir recomendações
    for model_name, rec in recommendations.items():
        print(f"\n🏗️ {model_name}")
        print(f"   🎯 Uso recomendado: {rec['uso_recomendado']}")
        
        print(f"   ✅ Vantagens:")
        for advantage in rec['vantagens']:
            print(f"      • {advantage}")
        
        print(f"   ⚠️ Desvantagens:")
        for disadvantage in rec['desvantagens']:
            print(f"      • {disadvantage}")
        
        print(f"   🎪 Cenários ideais:")
        for scenario in rec['cenarios']:
            print(f"      • {scenario}")
    
    # Salvar recomendações
    recommendations_path = os.path.join(project_config['paths']['outputs'], 'model_recommendations.json')
    with open(recommendations_path, 'w') as f:
        json.dump(recommendations, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Recomendações salvas em: {recommendations_path}")
    
    return recommendations

# Gerar recomendações
recommendations = generate_model_recommendations()

---

## 🎯 Resumo das Arquiteturas de Modelo

### ✅ Implementações Concluídas

1. **🏗️ U-Net Base Aprimorada**
   - Blocos de convolução com BatchNorm e Dropout
   - Conexões residuais opcionais
   - Configuração flexível de filtros

2. **⚡ Mecanismos de Atenção**
   - **Attention Gates**: Foco em regiões relevantes
   - **Self-Attention**: Captura dependências espaciais
   - **Channel Attention (SE)**: Atenção por canal

3. **🧠 Backbones Pré-treinados**
   - ResNet50 com weights do ImageNet
   - EfficientNet-B0 otimizado
   - DenseNet121 para features densas

4. **🎯 Variantes Avançadas**
   - U-Net++ (Nested U-Net) com múltiplos paths
   - Supervisão profunda (deep supervision)
   - Configurações otimizadas

5. **📊 Análise Completa**
   - Comparação de complexidade
   - Testes de inferência
   - Recomendações de uso

### 📈 Principais Características

- **🔧 Modularidade**: Blocos reutilizáveis e configuráveis
- **⚡ Performance**: Otimizado para GPU com mixed precision
- **🎯 Flexibilidade**: Múltiplas opções arquiteturais
- **📊 Análise**: Comparação detalhada de modelos
- **💾 Persistência**: Arquiteturas salvas para reutilização

### 🏆 Modelos Recomendados por Cenário

| Cenário | Modelo Recomendado | Justificativa |
|---------|-------------------|---------------|
| **Prototipagem** | U-Net Básica | Simplicidade e rapidez |
| **Produção Clínica** | U-Net + Atenção | Equilíbrio performance/custo |
| **Máxima Precisão** | U-Net + Atenção + Residual | Performance superior |
| **Transfer Learning** | U-Net + ResNet50 | Features pré-treinadas |
| **Deploy Eficiente** | U-Net + EfficientNet | Otimização de recursos |
| **Pesquisa Avançada** | U-Net++ | Estado da arte |

### 🚀 Próximos Passos

1. **📈 Loss Functions**: Execute `04_Loss_Functions_and_Metrics.ipynb`
2. **🚀 Training Pipeline**: Execute `05_Training_Pipeline.ipynb`
3. **📋 Model Evaluation**: Execute `06_Model_Evaluation.ipynb`

---

**✨ Arquiteturas implementadas e prontas para treinamento!**